In [542]:
from dotenv import load_dotenv
import os

# Common data processing
import json
import textwrap

# Langchain
from langchain_community.graphs import Neo4jGraph
from langchain_community.vectorstores import Neo4jVector
from langchain_community.document_loaders import PyPDFLoader
from langchain_openai import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.chains import RetrievalQAWithSourcesChain
from langchain_openai import ChatOpenAI


# Warning control
import warnings
warnings.filterwarnings("ignore")

In [543]:
load_dotenv('.env', override=True)
NEO4J_URI = os.getenv('NEO4J_URI')
NEO4J_USERNAME = os.getenv('NEO4J_USERNAME')
NEO4J_PASSWORD = os.getenv('NEO4J_PASSWORD')
NEO4J_DATABASE = os.getenv('NEO4J_DATABASE') or 'neo4j'

OPENAI_API_KEY = os.getenv('AZURE_OPENAI_API_KEY')
OPENAI_ENDPOINT = os.getenv('AZURE_OPENAI_ENDPOINT')

In [544]:
# Instantiate Neo4j

kg = Neo4jGraph(
    url=NEO4J_URI, username=NEO4J_USERNAME, password=NEO4J_PASSWORD, database=NEO4J_DATABASE
)

In [545]:
from langchain_openai import AzureChatOpenAI, AzureOpenAIEmbeddings

def get_llm():
    load_dotenv('.env', override=True)
    return AzureChatOpenAI(azure_deployment="gpt-4o-mini", api_version="2024-08-01-preview")

def get_embedding():
    load_dotenv('.env', override=True)
    return AzureOpenAIEmbeddings(azure_deployment="text-embedding-ada-002", api_version="2023-05-15")

embedding_model = get_embedding()
llm_model = get_llm()

In [546]:
def pdf_loader(file_path: str):
    loader = PyPDFLoader(file_path)
    text = ""
    for page in loader.load():
        text += page.page_content
    return text

In [547]:
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 2000,
    chunk_overlap  = 200,
    length_function = len,
    is_separator_regex = False,
)

## Extract metadata for Chunks

In [548]:
import requests
import xmltodict
from time import sleep

url = "https://www.boe.es/datosabiertos/api/boe/sumario/20240101"


def remove_duplicated_from_list_dict(dict_list):
    unique_list = []
    for d in dict_list:
        if d not in unique_list:
            unique_list.append(d)
    return unique_list


def get_items(departament):
        if type(departament["epigrafe"]) == dict:
            if type(departament["epigrafe"]["item"]) == list:
                for item in departament["epigrafe"]["item"]:
                    yield item, departament["epigrafe"]["@nombre"], departament["@nombre"]
            else:
                yield departament["epigrafe"]["item"], departament["epigrafe"]["@nombre"], departament["@nombre"]
        else:
            for epigraf in departament["epigrafe"]:
                if type(epigraf["item"]) == list:
                    for item in epigraf["item"]:
                        yield item, epigraf["@nombre"], departament["@nombre"]
                else:
                    yield epigraf["item"], epigraf["@nombre"], departament["@nombre"]


def get_metadata(url: str):
    headers = {"Accept": "application/xml"}
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        data_dict = xmltodict.parse(response.content)
        chunks_with_metadata = []
        items_with_metadata = []
        epigraf_with_metadata = []
        department_with_metadata = []
        seccion_with_metadata = []
        sumario_with_metadata = []
        sumario = data_dict["response"]["data"]["sumario"]["diario"]["sumario_diario"]
        sumario_with_metadata.append({"sumario_id": sumario["identificador"], "source": sumario["url_pdf"]["#text"]})
        for seccion in data_dict["response"]["data"]["sumario"]["diario"]["seccion"]:
            seccion_with_metadata.append({'seccion_id': seccion["@nombre"], "sumario_id": sumario["identificador"]})
            for departament in seccion["departamento"]:
                department_with_metadata.append({'department_id': departament["@nombre"], 'seccion_id': seccion["@nombre"], "sumario_id": sumario["identificador"]})
                for item, epigraf, department in get_items(departament):
                    epigraf_with_metadata.append({'epigraf_id': epigraf, 'department_id': department, 'seccion_id': seccion["@nombre"], "sumario_id": sumario["identificador"]})
                    items_with_metadata.append(
                        {
                            'title': item["titulo"],
                            'item_id': item["identificador"],
                            'epigraf_id': epigraf,
                            'department_id': department,
                            'seccion_id': seccion["@nombre"],
                            'sumario_id': sumario["identificador"],
                            'control': item["control"],
                            'source': item["url_pdf"]["#text"]
                        }
                    )
                    item_text = pdf_loader(file_path=f"C:/Users/2373225/projects/genai-1/data/pdfs/{item['identificador']}.pdf")
                    item_text_chunks = text_splitter.split_text(item_text)
                    chunk_seq_id = 0
                    for chunk in item_text_chunks:
                        chunks_with_metadata.append(
                            {
                                'text': chunk,
                                'chunk_id': f"{item['identificador']}-chunk{chunk_seq_id:04d}",
                                'item_id': item["identificador"],
                                'epigraf_id': epigraf,
                                'department_id': department,
                                'seccion_id': seccion["@nombre"],
                                'sumario_id': sumario["identificador"],
                                'chunkSeqId': chunk_seq_id,
                                'source': item["url_pdf"]["#text"]
                            }
                        )
                        chunk_seq_id += 1

    return (remove_duplicated_from_list_dict(chunks_with_metadata), 
           remove_duplicated_from_list_dict(items_with_metadata), 
           remove_duplicated_from_list_dict(epigraf_with_metadata), 
           remove_duplicated_from_list_dict(department_with_metadata), 
           remove_duplicated_from_list_dict(seccion_with_metadata), 
           remove_duplicated_from_list_dict(sumario_with_metadata))


chunks_metadata, items_metadata, epigraf_metadata, department_metadata, seccion_metadata, sumario_metadata = get_metadata(url=url)

## Create all nodes

#### 1. Chunk nodes

In [549]:
merge_chunk_node_query = """
MERGE(mergedChunk:Chunk {chunk_id: $chunkParam.chunk_id})
    ON CREATE SET 
        mergedChunk.text = $chunkParam.text,
        mergedChunk.item_id = $chunkParam.item_id,
        mergedChunk.epigraf_id = $chunkParam.epigraf_id,
        mergedChunk.department_id = $chunkParam.epigraf_id,
        mergedChunk.seccion_id = $chunkParam.seccion_id,
        mergedChunk.sumario_id = $chunkParam.sumario_id,
        mergedChunk.source = $chunkParam.source, 
        mergedChunk.chunkSeqId = $chunkParam.chunkSeqId
        
RETURN mergedChunk
"""

kg.query(merge_chunk_node_query, params={'chunkParam': chunks_metadata[0]})

[{'mergedChunk': {'seccion_id': 'II. Autoridades y personal. - A. Nombramientos, situaciones e incidencias',
   'textEmbedding': [-0.025936845690011978,
    -0.032542891800403595,
    -0.006318383384495974,
    -0.018532119691371918,
    -0.04367028549313545,
    -0.00758747523650527,
    -0.006426679436117411,
    0.00944204069674015,
    0.0016396662686020136,
    -0.010112120769917965,
    0.016988905146718025,
    0.00264140241779387,
    -0.002042391337454319,
    0.017476236447691917,
    -0.034654658287763596,
    0.007580706384032965,
    0.017882345244288445,
    -0.012887201271951199,
    0.021618550643324852,
    -0.007316735573112965,
    0.0026837054174393415,
    0.01750330999493599,
    -0.019222505390644073,
    -0.004429975524544716,
    0.005898737348616123,
    0.0135099021717906,
    0.024569611996412277,
    -0.014877136796712875,
    0.027520671486854553,
    -0.007905594073235989,
    0.009225449524819851,
    -0.012020834721624851,
    0.004622877109795809,
    

In [550]:
kg.query("""
CREATE CONSTRAINT unique_chunk IF NOT EXISTS 
    FOR (c:Chunk) REQUIRE c.chunk_id IS UNIQUE
""")

[]

In [551]:
node_count = 0
for chunk in chunks_metadata:
    print(f"Creating `:Chunk` node for chunk ID {chunk['chunk_id']}")
    kg.query(merge_chunk_node_query, 
            params={
                'chunkParam': chunk
            })
    node_count += 1
print(f"Created {node_count} nodes")

Creating `:Chunk` node for chunk ID BOE-A-2024-1-chunk0000
Creating `:Chunk` node for chunk ID BOE-A-2024-1-chunk0001
Creating `:Chunk` node for chunk ID BOE-A-2024-2-chunk0000
Creating `:Chunk` node for chunk ID BOE-A-2024-3-chunk0000
Creating `:Chunk` node for chunk ID BOE-A-2024-3-chunk0001
Creating `:Chunk` node for chunk ID BOE-A-2024-4-chunk0000
Creating `:Chunk` node for chunk ID BOE-A-2024-4-chunk0001
Creating `:Chunk` node for chunk ID BOE-A-2024-5-chunk0000
Creating `:Chunk` node for chunk ID BOE-A-2024-5-chunk0001
Creating `:Chunk` node for chunk ID BOE-A-2024-6-chunk0000
Creating `:Chunk` node for chunk ID BOE-A-2024-6-chunk0001
Creating `:Chunk` node for chunk ID BOE-A-2024-6-chunk0002
Creating `:Chunk` node for chunk ID BOE-A-2024-6-chunk0003
Creating `:Chunk` node for chunk ID BOE-A-2024-7-chunk0000
Creating `:Chunk` node for chunk ID BOE-A-2024-7-chunk0001
Creating `:Chunk` node for chunk ID BOE-A-2024-7-chunk0002
Creating `:Chunk` node for chunk ID BOE-A-2024-7-chunk00

BufferError: Existing exports of data: object cannot be re-sized

#### 2. Item nodes

In [ ]:
kg.query("""
CREATE CONSTRAINT unique_item IF NOT EXISTS 
    FOR (i:Item) REQUIRE i.item_id IS UNIQUE
""")

[]

In [ ]:
merge_item_node_query = """
MERGE(mergedItem:Item {item_id:$ItemParam.item_id})
    ON CREATE SET 
        mergedItem.title = $ItemParam.title,
        mergedItem.item_id = $ItemParam.item_id,
        mergedItem.epigraf_id = $ItemParam.epigraf_id,
        mergedItem.department_id = $ItemParam.department_id,
        mergedItem.seccion_id = $ItemParam.seccion_id,
        mergedItem.sumario_id = $ItemParam.sumario_id,
        mergedItem.control = $ItemParam.control,
        mergedItem.source = $ItemParam.source
RETURN mergedItem
"""

node_count = 0
for item in items_metadata:
    print(f"Creating `:Item` node for item_id {item['item_id']}")
    kg.query(merge_item_node_query, 
            params={
                'ItemParam': item
            })
    node_count += 1
print(f"Created {node_count} nodes")

Creating `:Item` node for item_id BOE-A-2024-1
Creating `:Item` node for item_id BOE-A-2024-2
Creating `:Item` node for item_id BOE-A-2024-3
Creating `:Item` node for item_id BOE-A-2024-4
Creating `:Item` node for item_id BOE-A-2024-5
Creating `:Item` node for item_id BOE-A-2024-6
Creating `:Item` node for item_id BOE-A-2024-7
Creating `:Item` node for item_id BOE-A-2024-8
Creating `:Item` node for item_id BOE-A-2024-9
Creating `:Item` node for item_id BOE-A-2024-10
Creating `:Item` node for item_id BOE-A-2024-11
Creating `:Item` node for item_id BOE-A-2024-12
Creating `:Item` node for item_id BOE-A-2024-13
Creating `:Item` node for item_id BOE-A-2024-14
Creating `:Item` node for item_id BOE-A-2024-15
Creating `:Item` node for item_id BOE-A-2024-16
Creating `:Item` node for item_id BOE-A-2024-17
Creating `:Item` node for item_id BOE-A-2024-18
Creating `:Item` node for item_id BOE-A-2024-19
Creating `:Item` node for item_id BOE-A-2024-20
Creating `:Item` node for item_id BOE-A-2024-21
C

## Create Vector Index

In [ ]:
kg.query("""
         CREATE VECTOR INDEX `item_chunks` IF NOT EXISTS
          FOR (c:Chunk) ON (c.textEmbedding) 
          OPTIONS { indexConfig: {
            `vector.dimensions`: 1536,
            `vector.similarity_function`: 'cosine'    
         }}
""")

[]

In [ ]:
kg.query("""
         CREATE VECTOR INDEX `item_titles` IF NOT EXISTS
          FOR (i:Item) ON (i.titleEmbedding) 
          OPTIONS { indexConfig: {
            `vector.dimensions`: 1536,
            `vector.similarity_function`: 'cosine'    
         }}
""")

[]

## Embed text of chunk nodes and title of item nodes

In [ ]:
from time import sleep

# Total chunks to embed
total_nodes_to_embed = kg.query("""
    MATCH (chunk:Chunk)
    RETURN count(chunk) AS TotalChunksToEmbed
""")

# Function to get the number of nodes pending to embed
def pending_nodes_to_emded():
    number_embedded_nodes = kg.query(
      """
      MATCH (chunk:Chunk) WHERE chunk.textEmbedding IS NULL 
      RETURN count(chunk) AS NotEmbedChunks                          
      """
    )
    return number_embedded_nodes[0]["NotEmbedChunks"]

# Initialize pending_nodes variable
pending_nodes = pending_nodes_to_emded()

while pending_nodes != 0:
  try:
    print("Embedding nodes...")
    kg.query("""
        MATCH (chunk:Chunk WHERE chunk.textEmbedding IS NULL)
        WITH chunk
        LIMIT $embedding_batch_size
        WITH chunk, genai.vector.encode(
          chunk.text, 
          "AzureOpenAI", 
          {
            token: $openAiApiKey,
            resource: "knowledge-graphs",
            deployment: "text-embedding-ada-002"
          }) AS vector
        CALL db.create.setNodeVectorProperty(chunk, "textEmbedding", vector)
        """, 
        params={"openAiApiKey": OPENAI_API_KEY, "embedding_batch_size": 50} 
        )
    
    pending_nodes = pending_nodes_to_emded()
    print(f"Number of pending nodes to embed: {pending_nodes}")
    sleep(60)
  except:
    print("Embedding has reach the limit rate per minut")
    sleep(60)
    continue

print("All nodes have been embeded!")

Embedding nodes...
Number of pending nodes to embed: 469
Embedding nodes...
Number of pending nodes to embed: 419
Embedding nodes...
Number of pending nodes to embed: 369
Embedding nodes...
Number of pending nodes to embed: 319
Embedding nodes...
Number of pending nodes to embed: 269
Embedding nodes...
Number of pending nodes to embed: 219
Embedding nodes...
Number of pending nodes to embed: 169
Embedding nodes...
Number of pending nodes to embed: 119
Embedding nodes...
Number of pending nodes to embed: 69
Embedding nodes...
Number of pending nodes to embed: 19
Embedding nodes...
Number of pending nodes to embed: 0
All nodes have been embeded!


In [ ]:
total_nodes_to_embed = kg.query("""
    MATCH (item:Item)
    RETURN count(item) AS TotalItemsToEmbed
""")

# Function to get the number of nodes pending to embed
def pending_nodes_to_emded():
    number_embedded_nodes = kg.query(
      """
      MATCH (item:Item) WHERE item.titleEmbedding IS NULL 
      RETURN count(item) AS NotEmbedItems                          
      """
    )
    return number_embedded_nodes[0]["NotEmbedItems"]

# Initialize pending_nodes variable
pending_nodes = pending_nodes_to_emded()

while pending_nodes != 0:
  try:
    print("Embedding nodes...")
    kg.query("""
        MATCH (item:Item WHERE item.titleEmbedding IS NULL)
        WITH item
        LIMIT $embedding_batch_size
        WITH item, genai.vector.encode(
          item.title, 
          "AzureOpenAI", 
          {
            token: $openAiApiKey,
            resource: "knowledge-graphs",
            deployment: "text-embedding-ada-002"
          }) AS vector
        CALL db.create.setNodeVectorProperty(item, "titleEmbedding", vector)
        """, 
        params={"openAiApiKey": OPENAI_API_KEY, "embedding_batch_size": 50} 
        )
    
    pending_nodes = pending_nodes_to_emded()
    print(f"Number of pending nodes to embed: {pending_nodes}")
    if pending_nodes != 0:
        sleep(60)
  except Exception as e:
    print(e)
    sleep(60)
    continue

print("All nodes have been embeded!")

Embedding nodes...
Number of pending nodes to embed: 36
Embedding nodes...
Number of pending nodes to embed: 0
All nodes have been embeded!


## Use similarity search to find relevant chunks

In [ ]:
def neo4j_vector_search(question):
  """Search for similar nodes using the Neo4j vector index"""
  vector_search_query = """
    WITH genai.vector.encode(
      $question, 
      "AzureOpenAI", 
      {
        token: $openAiApiKey,
        resource: "knowledge-graphs",
        deployment: "text-embedding-ada-002"
      }) AS question_embedding
    CALL db.index.vector.queryNodes($index_name, $top_k, question_embedding) yield node, score
    RETURN score, node.title AS title, node.item_id AS boe_id
  """
  similar = kg.query(vector_search_query, 
                     params={
                      'question': question, 
                      'openAiApiKey':OPENAI_API_KEY,
                      'index_name': "item_titles", 
                      'top_k': 10})
  return similar

In [ ]:
search_results = neo4j_vector_search('Quiero información sobre la Resolución 1A0/38511/2023')

# RAG without connections. It's the same as standard RAG pipeline

In [ ]:
neo4j_vector_store = Neo4jVector.from_existing_index(
    embedding=embedding_model,
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    index_name="item_chunks",
    node_label="Chunk",
    text_node_property="text",
    embedding_node_property="textEmbedding"
)

retriever = neo4j_vector_store.as_retriever(search_kwargs={'k': 5})

chain = RetrievalQAWithSourcesChain.from_chain_type(
    llm_model, 
    chain_type="stuff", 
    retriever=retriever
)

# Create connections between items and chunks

### 1. NEXT connection

In [ ]:
for i in range(86):
  cypher = """
    MATCH (from_same_item:Chunk)
      WHERE from_same_item.item_id = $ItemIdParam
    WITH from_same_item
      ORDER BY from_same_item.chunkSeqId ASC
    WITH collect(from_same_item) as item_chunk_list
      CALL apoc.nodes.link(
        item_chunk_list,
        "NEXT",
        {avoidDuplicates: true}
      )
    RETURN size(item_chunk_list)
  """

  kg.query(cypher, params={'ItemIdParam': items_metadata[i]['item_id']})

  print(f"NEXT connections from chunks of item_id {items_metadata[i]['item_id']} created.")

NEXT connections from chunks of item_id BOE-A-2024-1 created.
NEXT connections from chunks of item_id BOE-A-2024-2 created.
NEXT connections from chunks of item_id BOE-A-2024-3 created.
NEXT connections from chunks of item_id BOE-A-2024-4 created.
NEXT connections from chunks of item_id BOE-A-2024-5 created.
NEXT connections from chunks of item_id BOE-A-2024-6 created.
NEXT connections from chunks of item_id BOE-A-2024-7 created.
NEXT connections from chunks of item_id BOE-A-2024-8 created.
NEXT connections from chunks of item_id BOE-A-2024-9 created.
NEXT connections from chunks of item_id BOE-A-2024-10 created.
NEXT connections from chunks of item_id BOE-A-2024-11 created.
NEXT connections from chunks of item_id BOE-A-2024-12 created.
NEXT connections from chunks of item_id BOE-A-2024-13 created.
NEXT connections from chunks of item_id BOE-A-2024-14 created.
NEXT connections from chunks of item_id BOE-A-2024-15 created.
NEXT connections from chunks of item_id BOE-A-2024-16 created.
N

### 2. PART_OF connection

In [ ]:
cypher = """
  MATCH (c:Chunk), (i:Item)
    WHERE c.item_id = i.item_id
  MERGE (c)-[newRelationship:PART_OF]->(i)
  RETURN count(newRelationship)
"""

kg.query(cypher)

[{'count(newRelationship)': 519}]

### 3. SECTION connection

In [ ]:
cypher = """
  MATCH (first:Chunk), (i:Item)
  WHERE first.item_id = i.item_id
    AND first.chunkSeqId = 0
  WITH first, i
    MERGE (i)-[r:SECTION]->(first)
  RETURN count(r)
"""

kg.query(cypher)

[{'count(r)': 86}]

## RAG with connections and window retriever

In [567]:
retrieval_query_window = """
MATCH window=
    (:Chunk)-[:NEXT*0..2]->(node)-[:NEXT*0..2]->(:Chunk)
WITH node, score, window as longestWindow 
  ORDER BY length(window) DESC LIMIT 1
WITH nodes(longestWindow) as chunkList, node, score
  UNWIND chunkList as chunkRows
WITH collect(chunkRows.text) as textList, node, score
RETURN apoc.text.join(textList, " \n ") as text,
    score,
    node {.source} AS metadata
"""

In [568]:
vector_store_window = Neo4jVector.from_existing_index(
    embedding=embedding_model,
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    index_name="item_chunks",
    node_label="Chunk",
    text_node_property="text",
    retrieval_query=retrieval_query_window
)

# Create a retriever from the vector store
retriever_window = vector_store_window.as_retriever(search_kwargs={'k': 1})

# Create a chatbot Question & Answer chain from the retriever
chain_window = RetrievalQAWithSourcesChain.from_chain_type(
    llm_model, 
    chain_type="stuff", 
    retriever=retriever_window
)

In [ ]:
from rich.console import Console
from rich.markdown import Markdown

def rich_output(response):
    console = Console()
    md = Markdown(response)
    console.print(md)

In [ ]:
def print_comparison(question):
    response_a = chain({"question": question}, return_only_outputs=True,)
    response_b = chain_window({"question": question}, return_only_outputs=True,)    

    response_rag = ''
    response_rag += 'Response with standard RAG pipeline\n'
    response_rag += '-----------------------------------\n'
    response_rag += response_a["answer"]

    rich_output(response_rag)

    response_graphrag = ''
    response_graphrag += 'Response with GRAPH-RAG pipeline\n'
    response_graphrag += '-----------------------------------\n'
    response_graphrag += response_b["answer"]

    rich_output(response_graphrag)

## Example 1: Ask questions contained in the documents

In [565]:
# Define a question
question = "Quiero información sobre la Resolución 1A0/38511/2023, de 15 de noviembre, del Centro Criptológico Nacional"

print_comparison(question)

Response with standard RAG pipeline                                        

La Resolución 1A0/38511/2023, de 15 de noviembre, del Centro Criptológico Nacional certifica la seguridad del      
producto «TC-FNMT versión 5.6», solicitado por la Fábrica Nacional de Moneda y Timbre-Real Casa de la Moneda. Esta 
certificación se basa en la evaluación de seguridad conforme a las normas de evaluación de seguridad de tecnologías
de la información y se encuentra sujeta al Reglamento de Evaluación y Certificación de la Seguridad de las         
Tecnologías de la Información.                                                                                     

La resolución establece que el producto cumple con los requisitos de seguridad especificados en la Declaración de  
Seguridad correspondiente y detalla los perfiles de protección aplicables. La certificación entrará en vigor al día
siguiente de su publicación en el Boletín Oficial del Estado.

Response with GRAPHRAG pipeline                                          

La Resolución 1A0/38511/2023, de 15 de noviembre, del Centro Criptológico Nacional, certifica la seguridad del     
producto «TC-FNMT versión 5.6», solicitado por la Fábrica Nacional de Moneda y Timbre-Real Casa de la Moneda. Esta 
resolución se basa en la evaluación de la seguridad del producto conforme a las normas establecidas y se encuadra  
dentro del marco regulador del Centro Criptológico Nacional. La certificación asegura que el producto cumple con   
los requisitos especificados en su Declaración de Seguridad.

In [569]:
# Define a question
question = "Indica y explica los 5 puntos sobre la Resolución 1A0/38511/2023, de 15 de noviembre, del Centro Criptológico Nacional"

print_comparison(question)

Response with standard RAG pipeline                                        

No sé.

Response with GRAPHRAG pipeline                                          

 1 La Resolución 1A0/38511/2023, emitida el 15 de noviembre por el Centro Criptológico Nacional, certifica la      
   seguridad del producto «TC-FNMT versión 5.6», solicitado por la Fábrica Nacional de Moneda y Timbre-Real Casa de
   la Moneda.                                                                                                      
 2 Esta certificación se basa en un informe técnico de evaluación que verifica que el producto cumple con las      
   propiedades de seguridad especificadas en su Declaración de Seguridad.                                          
 3 La evaluación se llevó a cabo conforme a las normas de la «Common Methodology for Information Technology        
   Security Evaluation» y se le otorgó un nivel de garantía de evaluación EAL4, incluyendo varios componentes      
   adicionales de evaluación.                                                                                      
 4 La certificación, su alcance y vigencia, así como el uso del producto como certificado, están sujetos a lo      
   estipulado en el Reglamento de Evaluación y Certificación de la Seguridad de las Tecnologías de la Información. 
 5 La resolución entrará en vigor al día siguiente de su publicación en el «Boletín Oficial del Estado» y los      
   documentos relevantes estarán disponibles para consulta en el Centro Criptológico Nacional.

## Retrieve chunks creating a window from the item title and getting chunk context

In [ ]:
retrieval_query_window_for_item_title = """
MATCH window=
    (node)-[:SECTION*0..1]->(:Chunk)-[:NEXT*0..1]->(:Chunk)
WITH node, score, window as longestWindow 
  ORDER BY length(window) DESC LIMIT 6
WITH nodes(longestWindow) as itemList, node, score
  UNWIND itemList as itemRows
WITH collect(itemRows.text) as textList, node, score
RETURN apoc.text.join(textList, " \n ") as text, score, node {.source} AS metadata
"""

In [ ]:
vector_store_window = Neo4jVector.from_existing_index(
    embedding=embedding_model,
    url=NEO4J_URI,
    username=NEO4J_USERNAME,
    password=NEO4J_PASSWORD,
    database="neo4j",
    index_name="boe_item_title",
    text_node_property="title",
    retrieval_query=retrieval_query_window_for_item_title
)

# Create a retriever from the vector store
retriever_window = vector_store_window.as_retriever(search_kwargs={'k': 10})

# Create a chatbot Question & Answer chain from the retriever
chain_window = RetrievalQAWithSourcesChain.from_chain_type(
    llm_model, 
    chain_type="stuff", 
    retriever=retriever_window
)